In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.neural_network import MLPClassifier#cuidado
from sklearn.datasets import fetch_openml
import time
from tqdm import tqdm
import sys

<h1>cargar y escalar datos</h1>

In [2]:
X,y=fetch_openml(data_id=1560,return_X_y=True)#cargo de openml
print("en x:")
print(X)
print("en y:")
print(y)

en x:
        V1      V2      V3     V4     V5   V6   V7   V8    V9  V10  ...  V26   
0     23.0   240.0   357.0  120.0  120.0  0.0  0.0  0.0  73.0  0.5  ...  0.0  \
1     45.0     5.0   632.0  132.0  132.0  4.0  0.0  4.0  17.0  2.1  ...  0.0   
2     45.0   177.0   779.0  133.0  133.0  2.0  0.0  5.0  16.0  2.1  ...  0.0   
3     45.0   411.0  1192.0  134.0  134.0  2.0  0.0  6.0  16.0  2.4  ...  0.0   
4     45.0   533.0  1147.0  132.0  132.0  4.0  0.0  5.0  16.0  2.4  ...  0.0   
...    ...     ...     ...    ...    ...  ...  ...  ...   ...  ...  ...  ...   
2121   3.0  2059.0  2867.0  140.0  140.0  0.0  0.0  6.0  79.0  0.2  ...  0.0   
2122   3.0  1576.0  2867.0  140.0  140.0  1.0  0.0  9.0  78.0  0.4  ...  0.0   
2123   3.0  1576.0  2596.0  140.0  140.0  1.0  0.0  7.0  79.0  0.4  ...  0.0   
2124   3.0  1576.0  3049.0  140.0  140.0  1.0  0.0  9.0  78.0  0.4  ...  0.0   
2125   3.0  2796.0  3415.0  142.0  142.0  1.0  1.0  5.0  74.0  0.4  ...  1.0   

      V27  V28  V29  V30  V31  V3

c:\Users\jmlozanoo\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [3]:

scaler=MinMaxScaler()


X=scaler.fit_transform(X)
print("en x despues:")
print(X)

X.shape, y.shape

en x despues:
[[0.46808511 0.07281553 0.02113527 ... 0.         1.         0.        ]
 [0.93617021 0.00151699 0.10416667 ... 0.         0.         0.        ]
 [0.93617021 0.05370146 0.14855072 ... 0.         0.         0.        ]
 ...
 [0.04255319 0.47815534 0.69716184 ... 0.         0.         0.        ]
 [0.04255319 0.47815534 0.8339372  ... 0.         0.         0.        ]
 [0.04255319 0.84830097 0.94444444 ... 0.         0.         0.        ]]


((2126, 35), (2126,))

<h1>algoritmo</h1>

In [4]:
resul=pd.DataFrame(index=range(5,31,5), columns=["Entrena","Verifica","T_Ejecuccion"])
#en index pones el rango de numeros
resul.index.name="Neuronas Ocultas"
resul
# de 5 a 51 incrementando en 5
# aqui seria 51
#para cada numero de neuronas que aciertos en entrena, verifica y que tiempo de ejecuccion

,Entrena,Verifica,T_Ejecuccion
Neuronas Ocultas,,,
5,NaN,NaN,NaN
10,NaN,NaN,NaN
15,NaN,NaN,NaN
20,NaN,NaN,NaN
25,NaN,NaN,NaN
30,NaN,NaN,NaN


In [5]:
K=10
skf=StratifiedKFold(n_splits=K)#defino el algoritmo

In [8]:
#para h en neuronas
    #mlp con neuronas
    #para validacion cruzada
        #entrenas con train
        #guardas puntuaciones
    #guardas medias


#FM FRR
#para cada numero de neuronas haces una validacion cruzada

for h in resul.index:
   
    r_fold=pd.DataFrame(columns=["Entrena","Verifica","T_Ejecuccion"])
    r_fold.index.name="Fold"
    k=0
    mlp=MLPClassifier(hidden_layer_sizes=(h),max_iter=1000)#maximas interaciones, y numero de ocultas



    for train_index, test_index in tqdm(skf.split(X,y),total=K, unit="Fold"):
        print("train:",train_index)
        print("test:",test_index)
        comienzo=time.time()#inicio tiempo
        mlp.fit(X[train_index],y[train_index])#entreno cogiendo los indices
        r_fold.loc[k,:]=[mlp.score(X[train_index],y[train_index]),mlp.score(X[test_index],y[test_index]),time.time()-comienzo]
        #fin tiempo
        k+=1
        print("r_fold:",r_fold)
        #añado cogiendo las puntuaciones 

    print("Tamano capa oculta:", h,"\n")
    display(r_fold)
    print("Media:\n ",r_fold.mean(),"\n")
    print("\nTiempo total (seg)=",r_fold.sum(axis=0)[2],"\n")
    resul.loc[h,:]=[r_fold.mean()[0],r_fold.mean()[1],r_fold.sum(axis=0)[2]]#media de cada columna y la ultima axis 0





  0%|          | 0/10 [00:00<?, ?Fold/s]

train: [ 190  191  192 ... 2123 2124 2125]
test: [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 195 198 199 200 201 202 203 204
 205 206 207 208 209 210 211 212 213 214 215 218 219 240 296]


 10%|█         | 1/10 [00:00<00:05,  1.71Fold/s]

r_fold:        Entrena  Verifica T_Ejecuccion
Fold                                 
0     0.993204  0.957746      0.57626
train: [   0    1    2 ... 2123 2124 2125]
test: [190 191 192 193 194 196 197 216 217 220 221 222 223 224 225 226 227 228
 229 230 231 232 233 234 235 236 237 238 239 241 242 243 244 245 246 247
 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265
 266 267 268 269 270 271 272 273 274 275 276 277 278 279 280 281 282 283
 284 285 286 287 288 289 290 291 292 293 294 295 297 298 299 300 301 309
 317 319 320 323 324 328 334 335 339 340 341 342 351 352 357 359 363 371
 377 380 381 382 383 384 385 386 387 388 389 390 391 393 394 396 401 413
 414 415 425 426 427 428 429 430 431 432 433 440 441 448 449 450 451 452
 453 454 455 456 457 458 459 460 462 463 464 479 486 487 488 489 490 491
 492 493 494 495 496 497 498 499 500 501 502 503 504 505 506 507 508 509
 510 511 512 514 515 516 517 518 519 520 521 522 523 524 525 526 527 528
 529 530 531 532 535 536 5

 20%|██        | 2/10 [00:01<00:04,  2.00Fold/s]

...

r_fold:        Entrena  Verifica T_Ejecuccion
Fold                                 
0     0.995295  0.962441     0.566306
1     0.995818  0.985915     1.149017
2     0.994773  0.995305     1.097741
3     0.995295  0.985915     0.894737
4     0.995818  0.967136     1.040533
5     0.991636       1.0     0.638197
6     0.991641  0.995283     0.503647
7      0.99373  0.990566       0.6237
8     0.994253  0.990566     0.573699
train: [   0    1    2 ... 1960 1961 1962]
test: [1413 1429 1431 1445 1453 1455 1462 1464 1470 1473 1474 1475 1476 1490
 1527 1528 1546 1547 1548 1549 1600 1602 1604 1605 1918 1919 1920 1921
 1922 1923 1924 1925 1926 1928 1929 1930 1931 1932 1933 1934 1935 1936
 1937 1938 1939 1940 1941 1942 1943 1963 1964 1965 1966 1967 1968 1969
 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983
 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997
 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011
 2012 2013 2014 2015 2016 20

100%|██████████| 10/10 [00:07<00:00,  1.33Fold/s]

r_fold:        Entrena  Verifica T_Ejecuccion
Fold                                 
0     0.995295  0.962441     0.566306
1     0.995818  0.985915     1.149017
2     0.994773  0.995305     1.097741
3     0.995295  0.985915     0.894737
4     0.995818  0.967136     1.040533
5     0.991636       1.0     0.638197
6     0.991641  0.995283     0.503647
7      0.99373  0.990566       0.6237
8     0.994253  0.990566     0.573699
9     0.994253  0.966981     0.426151
Tamano capa oculta: 30 



,Entrena,Verifica,T_Ejecuccion
Fold,,,
0,0.995295,0.962441,0.566306
1,0.995818,0.985915,1.149017
2,0.994773,0.995305,1.097741
3,0.995295,0.985915,0.894737
4,0.995818,0.967136,1.040533
5,0.991636,1.0,0.638197
6,0.991641,0.995283,0.503647
7,0.99373,0.990566,0.6237
8,0.994253,0.990566,0.573699


Media:
  Entrena         0.994251
Verifica        0.984011
T_Ejecuccion    0.751373
dtype: object 


Tiempo total (seg)= 7.5137269496917725 



In [9]:
print("Resultado final:")
print(resul)


Resultado final:
                   Entrena  Verifica T_Ejecuccion
Neuronas Ocultas                                 
5                  0.99216  0.986839     5.154092
10                 0.99284  0.986832     5.200951
15                0.992579  0.983537     4.844489
20                0.993101  0.985424      4.68983
25                0.993781  0.983065     6.164099
30                0.994251  0.984011     7.513727


In [10]:
resul["Entrena"]=pd.to_numeric(resul["Entrena"])#convertirlo a numero
print("maximo:",resul["Entrena"].max())
print("tamano capa oculta:",resul["Entrena"].idxmax())#sacas maximo y su indice

resul["Verifica"]=pd.to_numeric(resul["Verifica"])
print("maximo:",resul["Verifica"].max())
print("tamano capa oculta:",resul["Verifica"].idxmax())

resul["T_Ejecuccion"]=pd.to_numeric(resul["T_Ejecuccion"])
print("minimo:",resul["T_Ejecuccion"].min())
print("tamano capa oculta:",resul["T_Ejecuccion"].idxmin())

maximo: 0.9942512348824873
tamano capa oculta: 30
maximo: 0.9868389582779697
tamano capa oculta: 5
minimo: 4.689829587936401
tamano capa oculta: 20
